In [ ]:
from Parameters import ModelParams
from Mass_Loss_Class import MassLoss
from Fractionation_Class import Fractionation
from Model_Data_Loader import ModelDataLoader
from Results_Handler import ResultsHandler
from Misc_Class import Misc

In [ ]:
%matplotlib widget

In [ ]:
def main():

    # Initialize parameters and classes
    params = ModelParams()
    base_path = '/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/CW/critical_WMF/gridmin295/'
    data_loader = ModelDataLoader(base_path)
    mass_loss = MassLoss(params)
    fractionation = Fractionation(params)
    misc = Misc(params)

    # Load data
    # m_planet = [0.25, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    m_planet = [0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    water_percent = [x / 10 for x in range(1, 81)]
    radii_list, mass_list, teq_list = data_loader.load_model_data(m_planet, water_percent)

    # Mass loss model
    results = mass_loss.compute_mass_loss_parameters(mass_list, radii_list, teq_list)

    all_results = []

    for planet_index, result in enumerate(results):
        m_planet = result['mass']
        r_planet = result['radius']
        Teq = result['Teq']
        REUV = result['REUV']
        cs = result['cs']
        Mdot = result['Mdot']
        RS_flow = result['RS_flow']
        rho_flow = result['rho_flow']
        rho_EUV = result['rho_EUV']

        # Other Radii
        R_b = misc.calculate_R_b(m_planet, cs)
        R_rcb = misc.calculate_R_rcb(m_planet, Teq)

        # Temperature and pressure at R_EUV
        T_REUV = fractionation.compute_T_REUV(cs)

        if Teq >= T_REUV:
            print(f"Excluded: T_eq ({Teq:.2f} K) > T_REUV ({T_REUV:.2f} K) for planet mass={m_planet/params.mearth:.2f} M_earth, radius={r_planet/params.rearth:.2f} R_earth.")
            continue

        P_EUV = misc.calculate_pressure_ideal_gas(rho_EUV, T_REUV)

        # Fractionation model
        b_i, mass_difference, flux_total, reservoir_ratio = fractionation.compute_fractionation_params(cs, REUV, Mdot)

        phi_O, phi_H, x_O = fractionation.iterative_fractionation(flux_total, REUV, m_planet, T_REUV, b_i, mass_difference, reservoir_ratio)
        print(f"Planet {planet_index} with mass={m_planet/params.mearth:.2f} M_earth results: phi_O = {phi_O}, phi_H = {phi_H}, x_O = {x_O}")

        # Gather all data in one list of dictionaries, one for each planet
        planet_results = {
            'm_planet': m_planet,
            'r_planet': r_planet,
            'Teq': Teq,
            'REUV': REUV,
            'cs': cs,
            'Mdot': Mdot,
            'RS_flow': RS_flow,
            'rho_flow': rho_flow,
            'rho_EUV': rho_EUV,
            'R_b': R_b,
            'R_rcb': R_rcb,
            'T_REUV': T_REUV,
            'P_EUV': P_EUV,
            'phi_H': phi_H,
            'phi_O': phi_O,
            'x_O': x_O
        }

        all_results.append(planet_results)

    # save and/or plot results
    # ResultsHandler.save_results(all_results, "fractionation_results.pkl")
        
    # ResultsHandler.plot_m_planet_r_planet(all_results)
    ResultsHandler.plot_cs_REUV(all_results)
    # ResultsHandler.plot_pressure_vs_m_planet(all_results)
    # ResultsHandler.plot_T_REUV_m_planet(all_results)
    ResultsHandler.plot_combined_T_P(all_results)
    # ResultsHandler.plot_REUV_r_planet_vs_m_planet(all_results)
    # ResultsHandler.plot_xO_phiO_vs_m_planet(all_results)
    ResultsHandler.plot_phiH_phiO_vs_REUV(all_results)
    ResultsHandler.plot_radii_comparison(all_results)

if __name__ == "__main__":
   main()